In [2]:
!echo $JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-11.0.10.jdk/Contents/Home'
!echo $JAVA_HOME

/Library/Java/JavaVirtualMachines/jdk-11.0.10.jdk/Contents/Home
/Library/Java/JavaVirtualMachines/jdk-11.0.10.jdk/Contents/Home


In [3]:

from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()



In [4]:

#IMPORTING SETUP PACKAGES
import statsmodels.formula.api as smf
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)



/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
number_of_top_items = '5000'
plot_name = number_of_top_items[0] + 'k'

#print(plot_name)

df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top'+number_of_top_items+'_seller_YTD_group_by_item_visit_20210226.pickle')


In [6]:
print(df.item_code.nunique())



4488


In [7]:
df.head()

,min_date,max_date,item_code,item_price,delivery_weeks,avg_sales,avg_unique_views,conv_rate,bin,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales,log_avg_unique_views,log_conv_rate
0,2020-01-01,2020-05-23,000000001000012530,99.99,1,0.520833,35.743056,0.014573,144,000000008000000774,ESSZIMMERSTÜHLE&BÄNKE,BÄNKE,BÄNKE,4.605071,0.000100,-0.652133,3.576359,-4.221842
1,2020-01-01,2020-01-02,000000001000012794,219.99,2,0.000000,27.000000,0.000001,2,000000008000000826,BETTEN,BETTGESTELLE,MASSIVHOLZBETTEN,5.393583,0.693197,-9.210340,3.295841,-9.210340
2,2020-01-01,2020-03-13,000000001000021922,129.99,2,1.589041,147.191781,0.010797,73,000000001000021922-P,BAD,BADSCHRÄNKE,WASCHBECKENUNTERSCHRÄNKE,4.867458,0.693197,0.463194,4.991737,-4.519385
3,2020-01-01,2020-01-01,000000001000022179,549.99,3,0.000000,50.000000,0.000001,1,000000008000001920,BETTEN,BETTGESTELLE,POLSTERBETTEN,6.309900,1.098646,-9.210340,3.912025,-9.210340
4,2020-01-01,2020-01-01,000000001000024320,99.99,1,0.000000,8.000000,0.000001,1,000000008000002133,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.605071,0.000100,-9.210340,2.079454,-9.210340


In [8]:
print(df.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg_sales                float64
avg_unique_views         float64
conv_rate                float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
log_avg_unique_views     float64
log_conv_rate            float64
dtype: object


In [9]:
def filter_df(DF = df,bins=6, avg_PDP = 19.):
    df_filt = DF[DF['bin'] > bins]
    df_items = df_filt.groupby('item_code').item_price.nunique() > 1
    df_filt  = df_filt.merge(df_items,on='item_code',how='inner')
    df_filt  = df_filt[df_filt['item_price_y']== True]
    #filter PDP
    #print(df_filt)
    df_filt = df_filt[df_filt['avg_unique_views'] > avg_PDP]
    #print(df_filt)
    return df_filt

In [10]:
df_filt = filter_df(df)


print(df_filt.item_code.nunique())


4181


In [11]:
#introducing the similarity score
sim_score = pd.read_parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/recommendations_prod/part-00000-5f77392a-0bfa-4f0b-a4ae-d816ce2d71de-c000.snappy.parquet',engine='fastparquet')

In [12]:
sim_score

,sku,recommended_sku,score
0,000000001000200072,000000001000200087,0.260000
1,000000001000200072,000000001000183538,0.216580
2,000000001000200072,000000001000142209,0.200000
3,000000001000200072,000000001000204906,0.190794
4,000000001000200072,000000001000183468,0.183980
...,...,...,...
3827600,000000001000064787,000000001000115010,0.203238
3827601,000000001000064787,000000001000112641,0.201339
3827602,000000001000064787,000000001000046204,0.200052
3827603,000000001000064787,000000001000114086,0.199867


In [13]:
# Filter table with only topX item_code

unique_item_codes = list(df_filt.item_code.unique())


sim_score_filt = sim_score[sim_score['sku'].isin(unique_item_codes)]

sim_score_filt

,sku,recommended_sku,score
780,000000001000135008,000000001000135012,0.881024
781,000000001000135008,000000001000121202,0.715546
782,000000001000135008,000000001000167468,0.633481
783,000000001000135008,000000001000166799,0.564164
784,000000001000135008,000000001000208980,0.544883
...,...,...,...
3827360,000000001000176744,000000001000171107,0.339302
3827361,000000001000176744,000000001000163202,0.335340
3827362,000000001000176744,000000001000163190,0.327887
3827363,000000001000176744,000000001000176438,0.324293


In [14]:
#Create a map with X subset for X linear fit:

def create_map(list_of_items = unique_item_codes , df=sim_score_filt, score_cut = 0.):
    map_of_items = {}
    for item_code in list_of_items:
        df_mod = df[df['sku'] == item_code]
        df_score_filt = df_mod[df_mod['score'] > score_cut]
        final_set_sku = list(df_score_filt.recommended_sku)
        final_set_sku.insert(0,item_code)
        map_of_items[item_code] = final_set_sku
    #print(map_of_items)
    return map_of_items


In [15]:
sim_score_bin = [0.16]#0.3,0.4,0.5,0.6,0.7,0.8,0.9]




In [16]:
map_of_map_of_items = {}
for sim_score_i in sim_score_bin:
    Map_of_Items = create_map(score_cut = sim_score_i)
    map_of_map_of_items[str(sim_score_i)] = Map_of_Items
#print(map_of_map_of_items)

In [17]:
#load greater dataset 
largest_dataset = '/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/All_catalogue_seller_YTD_group_by_item_visit_20210226.pickle'

#df_k = pd.read_pickle(dataset10k)
df_k = pd.read_pickle(largest_dataset)

In [18]:
PDP_bin = [19]#,30,40,50]

In [19]:
df_PDP_bin_filt = {}
for PDP_view in PDP_bin:
    df_k_filt = filter_df(df_k, avg_PDP= PDP_view)
    df_PDP_bin_filt[PDP_view] = df_k_filt
    #print(df_k_filt)

#print(df_PDP_bin_filt)


In [20]:
def create_map_of_dataframes(items_map = Map_of_Items,df=df_filt):
    map_of_df = {}
    for key in items_map.keys():
        items  = items_map[key]
        #print(f'Item_code = {key}')
        
        df_mod = df[df['item_code'].isin(items)]
        #
        #print(f'Number of items per cluster {df_mod.item_code.nunique()}')
        map_of_df[key]= df_mod
    return map_of_df



In [21]:
import numpy as np
def run_model(item_code, df, map_results):
    #print('\n')
    wls_cr_with_fe = smf.wls("conv_rate ~ log_price + log_delivery_weeks + C(item_code)", df, weights=df['bin']).fit()

    beta_log_price, beta_std_err_log_price = wls_cr_with_fe.params[-2].round(4),wls_cr_with_fe.bse[-2].round(4)
    beta_log_del, beta_std_err_log_del = wls_cr_with_fe.params[-1].round(4),wls_cr_with_fe.bse[-1].round(4)
    Result_log_price = 'Log_price  = ' + str(beta_log_price) + ' ± ' + str(beta_std_err_log_price)
    Result_log_del = 'Log_del_week = ' + str(beta_log_del) + ' ± ' + str(beta_std_err_log_del)
    flag_stat_ok = False
    if beta_log_price != 0 :
        flag_stat_ok = np.fabs((3*beta_std_err_log_price)/beta_log_price) < 1.
    #flag_stat_ok = fabs(beta_std_err_log_price/beta_log_price)

    if flag_stat_ok:
        #print(f'Enough statistics for the item = {item_code}')
        #print(Result_log_price)
        #print(Result_log_del)
        map_results[item_code] = [beta_log_price, beta_std_err_log_price, beta_log_del,beta_std_err_log_del, df.item_code.nunique(), True]
        #print(map_results[item_code])
        return True
    else:
        #print(f'Not enough statistics for the item = {item_code}')
        #print(Result_log_price)
        #print(Result_log_del)
        map_results[item_code] = [beta_log_price, beta_std_err_log_price, beta_log_del,beta_std_err_log_del, df.item_code.nunique(), False]
        #print(map_results[item_code])
        return False


In [22]:
df_result = pd.DataFrame()

map_of_maps_of_df = {}
#print('Cut on PDP, sim_score, result = ')
for PDP_cut in df_PDP_bin_filt.keys():
    for score_cut in  map_of_map_of_items.keys():
        map_of_dfs = create_map_of_dataframes(map_of_map_of_items[score_cut],df=df_PDP_bin_filt[PDP_cut])
        map_of_maps_of_df[PDP_cut] = map_of_dfs
        print(f'Number of items passing the PE cut {len(map_of_dfs)}')

        map_result = {}

        #print("Model used: conv_rate ~ log_price + log_delivery_weeks + C(item_code)\n")
        map_items_stat = {}
        for key in map_of_dfs.keys():
            #map_of_dfs[key].head()
            map_items_stat[key] = run_model(key, map_of_dfs[key], map_result)

            #print(map_result)

        items_stat_series = pd.Series(map_items_stat)
        #print(map_items_stat)
        #print(items_stat_series)


        
        x,y, y_err, rel_err = {'True':[],'False':[]},{'True':[],'False':[]},{'True':[],'False':[]},{'True':[],'False':[]}
        for i_code,flag in items_stat_series.items():
        #flag=T
            if flag == True:

                y['True'].append(map_result[i_code][0])
                y_err['True'].append(map_result[i_code][1])
                x['True'].append(map_result[i_code][-2])
                #print('True',map_result[i_code][0],map_result[i_code][1])
                rel_err['True'].append(abs(map_result[i_code][1]/map_result[i_code][0]))


            else:
                y['False'].append(map_result[i_code][0])
                y_err['False'].append(map_result[i_code][1])
                x['False'].append(map_result[i_code][-2])
                #print('False',map_result[i_code][0],map_result[i_code][1])
                rel_err['False'].append(abs(map_result[i_code][1]/map_result[i_code][0]))

                continue
        #print(f'{PDP_cut},{score_cut} , {len(x)}')
        df_result = pd.DataFrame.from_dict(map_result, orient='index', columns = ['PE', 'PE_err', 'log_del_week', 'log_del_week_err', 'Cluster_size', 'has_enough_stat'])
        #print (df_result.head(100))

Number of items passing the PE cut 4181


/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1578: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1578: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1578: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1578: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/statsmodel

In [23]:
df_result_true = df_result[df_result['has_enough_stat']==True]
print(len(df_result_true.index))


1907


In [24]:
df_result_false = df_result[df_result['has_enough_stat']==False]
items_false = list(df_result_false.index.values)
print(len(items_false))

2274


In [25]:
import pickle
def save_obj(obj, name ):
    with open('pickle_files/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [26]:
#df_result_true = df_result_true[['PE','PE_err']]
df_result_all = df_result[['PE','PE_err','has_enough_stat']]

In [27]:
#df_result_true = df_result_true.rename(column_index={'':'item_code'})
print(df_result_true.head(1000))
print(df_result_true.dtypes)

                        PE  PE_err  log_del_week  log_del_week_err  \
000000001000012530 -0.0187  0.0059       -0.0002            0.0004   
000000001000021922 -0.0194  0.0025       -0.0029            0.0005   
000000001000005727 -0.0246  0.0044       -0.0005            0.0005   
000000001000007327 -0.0229  0.0049       -0.0023            0.0008   
000000001000050787 -0.0284  0.0087       -0.0025            0.0017   
...                    ...     ...           ...               ...   
000000001000053282 -0.0138  0.0028       -0.0005            0.0002   
000000001000007033 -0.0054  0.0016       -0.0002            0.0003   
000000001000007443 -0.0217  0.0054        0.0000            0.0005   
000000001000008013 -0.0142  0.0038       -0.0007            0.0007   
000000001000008067 -0.0294  0.0048       -0.0013            0.0011   

                    Cluster_size  has_enough_stat  
000000001000012530            15             True  
000000001000021922            17             True  
000

In [28]:
#PREPARE THE CONVERSION RATE FILE
#final_items = list(df_result_true.index.values)
final_items = list(df_result_all.index.values)
print(len(final_items))



4181


In [29]:
print(df_filt.head(10))

      min_date    max_date           item_code  item_price_x  delivery_weeks  \
0   2020-01-01  2020-05-23  000000001000012530         99.99               1   
2   2020-05-31  2020-06-26  000000001000012530         99.99               1   
5   2020-08-06  2020-08-20  000000001000012530         99.99               1   
7   2020-09-16  2020-10-31  000000001000012530         99.99               1   
8   2020-11-01  2020-11-07  000000001000012530         99.99               1   
9   2020-11-10  2020-11-22  000000001000012530         99.99               1   
11  2020-12-08  2020-12-27  000000001000012530         99.99               1   
12  2021-01-05  2021-02-13  000000001000012530         99.99               1   
13  2020-01-01  2020-03-13  000000001000021922        129.99               2   
14  2020-04-15  2020-04-21  000000001000021922        119.99               4   

    avg_sales  avg_unique_views  conv_rate  bin item_parent_item_code  \
0    0.520833         35.743056   0.014573  14

In [30]:
df_final = df_filt[['item_code','max_date','item_price_x','conv_rate','avg_sales', 'avg_unique_views']]

print(df_final.head())

            item_code    max_date  item_price_x  conv_rate  avg_sales  \
0  000000001000012530  2020-05-23         99.99   0.014573   0.520833   
2  000000001000012530  2020-06-26         99.99   0.008923   0.444444   
5  000000001000012530  2020-08-20         99.99   0.026937   0.533333   
7  000000001000012530  2020-10-31         99.99   0.013050   0.478261   
8  000000001000012530  2020-11-07         99.99   0.021128   1.714286   

   avg_unique_views  
0         35.743056  
2         49.814815  
5         19.800000  
7         36.652174  
8         81.142857  


In [31]:
df_final= df_final[df_filt['item_code'].isin(final_items)]

In [32]:
print(df_final.head())

            item_code    max_date  item_price_x  conv_rate  avg_sales  \
0  000000001000012530  2020-05-23         99.99   0.014573   0.520833   
2  000000001000012530  2020-06-26         99.99   0.008923   0.444444   
5  000000001000012530  2020-08-20         99.99   0.026937   0.533333   
7  000000001000012530  2020-10-31         99.99   0.013050   0.478261   
8  000000001000012530  2020-11-07         99.99   0.021128   1.714286   

   avg_unique_views  
0         35.743056  
2         49.814815  
5         19.800000  
7         36.652174  
8         81.142857  


In [33]:
df_to_spark = df_final.groupby(['item_code']).nth(-1)
df_to_spark.reset_index(level=0,inplace=True)
df_to_spark.rename(columns={'index':'item_code'}, inplace=True)
spark_check = spark.createDataFrame(df_to_spark)


In [34]:
spark_check.show()

+------------------+----------+------------+--------------------+--------------------+------------------+
|         item_code|  max_date|item_price_x|           conv_rate|           avg_sales|  avg_unique_views|
+------------------+----------+------------+--------------------+--------------------+------------------+
|000000001000000145|2020-06-04|      169.99|0.019559359621451104|  1.3880597014925373| 70.97014925373135|
|000000001000002470|2021-02-25|       54.99| 0.10219078102189781|                 2.0|19.571428571428573|
|000000001000002471|2021-02-16|       53.99| 0.07605733802816901|              1.6875|           22.1875|
|000000001000002479|2021-02-08|       79.99| 0.04507437526205451|  3.3076923076923075| 73.38461538461539|
|000000001000002542|2021-01-31|       31.99| 0.12458012457912458|                 3.7|              29.7|
|000000001000003447|2021-02-04|     1699.99|0.005883352941176471| 0.11764705882352941|              20.0|
|000000001000004855|2021-02-04|      799.99|0.

In [35]:
spark_df = spark.createDataFrame(df_final)

In [36]:
spark_df.createOrReplaceTempView('Final_table')


In [37]:
spark_last_price = spark.sql("""with max_price as (SELECT item_code, ROW_NUMBER() over (PARTITION BY item_code ORDER BY max_date DESC) as rn, item_price_x, max_date
                            from Final_table)
                            SELECT item_code, item_price_x, max_date
                            from max_price
                            where rn=1
                            """)
spark_last_price.show()

+------------------+------------+----------+
|         item_code|item_price_x|  max_date|
+------------------+------------+----------+
|000000001000012530|       99.99|2021-02-13|
|000000001000012794|      259.99|2021-02-25|
|000000001000021922|      114.99|2021-02-16|
|000000001000022179|      479.99|2021-02-06|
|000000001000024320|      119.99|2020-12-01|
|000000001000034743|      279.99|2021-02-04|
|000000001000034762|      599.99|2021-02-04|
|000000001000119742|      349.99|2021-02-05|
|000000001000125896|      829.99|2021-01-30|
|000000001000127678|     1449.99|2021-02-25|
|000000001000145006|      289.99|2021-02-04|
|000000001000161399|      669.99|2021-02-19|
|000000001000199717|     1349.99|2021-02-25|
|000000001000200909|     1249.99|2021-02-25|
|000000001000005727|       99.99|2021-02-25|
|000000001000007327|      209.99|2021-02-25|
|000000001000007822|      349.99|2021-02-25|
|000000001000008870|      619.99|2021-02-11|
|000000001000016028|      139.99|2021-02-25|
|000000001

In [38]:
spark_last_price.createOrReplaceTempView('last_price')


In [39]:
spark_check.createOrReplaceTempView('check_last_price')


In [40]:
disjoint_spark = spark.sql("""SELECT *
                                from check_last_price as clp
                                inner join last_price as lp on lp.item_code = clp.item_code and lp.item_price_x!= clp.item_price_x  """)

disjoint_spark.show()

+---------+--------+------------+---------+---------+----------------+---------+------------+--------+
|item_code|max_date|item_price_x|conv_rate|avg_sales|avg_unique_views|item_code|item_price_x|max_date|
+---------+--------+------------+---------+---------+----------------+---------+------------+--------+
+---------+--------+------------+---------+---------+----------------+---------+------------+--------+



In [41]:
spark_filter = spark.sql("""SELECT ft.item_code, AVG(ft.conv_rate) as avg_conv_rate, MAX(ft.item_price_x) as last_price, MAX(ft.max_date) as last_date, AVG(avg_unique_views) as avg_unique_views
                            from Final_table as ft
                            inner join last_price as lp on lp.item_code = ft.item_code and lp.item_price_x = ft.item_price_x
                            GROUP BY ft.item_code
                            """)
spark_filter.show()

+------------------+--------------------+----------+----------+------------------+
|         item_code|       avg_conv_rate|last_price| last_date|  avg_unique_views|
+------------------+--------------------+----------+----------+------------------+
|000000001000200909|7.009834397539281E-4|   1249.99|2021-02-25|162.21114005042574|
|000000001000021922|0.023490245459702064|    114.99|2021-02-16| 90.11309523809524|
|000000001000119742|0.002189800317723...|    349.99|2021-02-05|        28.2109375|
|000000001000022179|0.003121343752816...|    479.99|2021-02-06| 93.25873910040576|
|000000001000012794|0.014916173111257461|    259.99|2021-02-25| 41.52597402597403|
|000000001000024320|0.024532024531024533|    119.99|2020-12-01| 22.35483870967742|
|000000001000125896| 0.00725756392830782|    829.99|2021-01-30| 71.68584656084656|
|000000001000145006|0.004873447305681728|    289.99|2021-02-04| 188.1904761904762|
|000000001000161399| 0.01291913289898707|    669.99|2021-02-19|27.065162037037037|
|000

In [42]:
spark_filter.createOrReplaceTempView('Final_table_filt')



In [43]:

#df_result_true.reset_index(level=0,inplace=True)
df_result_all.reset_index(level=0,inplace=True)



In [44]:
df_result_all.rename(columns={'index':'item_code'}, inplace=True)

/Users/gabriele.sabato/opt/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [45]:
print(df_result_all.head())

            item_code      PE  PE_err  has_enough_stat
0  000000001000012530 -0.0187  0.0059             True
1  000000001000021922 -0.0194  0.0025             True
2  000000001000034762 -0.0030  0.0023            False
3  000000001000125896 -0.0055  0.0038            False
4  000000001000199717 -0.0075  0.0049            False


In [46]:
spark_PE_df = spark.createDataFrame(df_result_all)

In [47]:
spark_PE_df.show()

+------------------+-------+------+---------------+
|         item_code|     PE|PE_err|has_enough_stat|
+------------------+-------+------+---------------+
|000000001000012530|-0.0187|0.0059|           true|
|000000001000021922|-0.0194|0.0025|           true|
|000000001000034762| -0.003|0.0023|          false|
|000000001000125896|-0.0055|0.0038|          false|
|000000001000199717|-0.0075|0.0049|          false|
|000000001000005727|-0.0246|0.0044|           true|
|000000001000007327|-0.0229|0.0049|           true|
|000000001000007822|-0.0241|0.0089|          false|
|000000001000016194|-0.0114|0.0086|          false|
|000000001000022565|-0.1456|0.1268|          false|
|000000001000050787|-0.0284|0.0087|           true|
|000000001000116614|-0.0067|0.0023|          false|
|000000001000117179|-0.0242|0.0072|           true|
|000000001000164660| -0.003|7.0E-4|           true|
|000000001000176734|-0.0149| 0.003|           true|
|000000001000178079|-0.0076|0.0049|          false|
|00000000100

In [48]:
spark_PE_df.createOrReplaceTempView('final_PE')

In [49]:
spark_final = spark.sql("""SELECT ftf.*, fpe.PE, fpe.PE_err, fpe.has_enough_stat
                            from Final_table_filt as ftf
                            INNER JOIN final_PE as fpe on fpe.item_code = ftf.item_code""")
spark_final.show()
spark_final.createOrReplaceTempView('spark_final_tmp')

+------------------+--------------------+----------+----------+------------------+-------+------+---------------+
|         item_code|       avg_conv_rate|last_price| last_date|  avg_unique_views|     PE|PE_err|has_enough_stat|
+------------------+--------------------+----------+----------+------------------+-------+------+---------------+
|000000001000012530|0.016219531350040728|     99.99|2021-02-13| 66.08138736263736|-0.0187|0.0059|           true|
|000000001000012794|0.014916173111257461|    259.99|2021-02-25| 41.52597402597403| -0.012|0.0052|          false|
|000000001000021922|0.023490245459702064|    114.99|2021-02-16| 90.11309523809524|-0.0194|0.0025|           true|
|000000001000022179|0.003121343752816...|    479.99|2021-02-06| 93.25873910040576|-0.0136|0.0033|           true|
|000000001000024320|0.024532024531024533|    119.99|2020-12-01| 22.35483870967742| -0.022|0.0097|          false|
|000000001000034743|0.005481390945972513|    279.99|2021-02-04|110.20249433106576|-0.018

In [50]:
spark.sql("""SELECT COUNT(DISTINCT item_code)
from spark_final_tmp""").show()


+-------------------------+
|count(DISTINCT item_code)|
+-------------------------+
|                     4181|
+-------------------------+



In [51]:
spark.sql("""SELECT COUNT(item_code)
from spark_final_tmp""").show()



+----------------+
|count(item_code)|
+----------------+
|            4181|
+----------------+



In [52]:
#Introduce d_item table
sp_df_item = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*.parquet',
    header=True)
sp_df_item.show()

+---------------+------------------+---------------------+-------------+----------------+----------+-------------+-----------+----------+-----------+--------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+------------------+-------------------+-------------------+---------+--------------------------------+---------------------------------------------+-----------------------+---------------------+------------------------+--------------------------+------------------------+--------------------+-----------+-------------------+------------------+-------------------+----------------------------+---------------+-----------------------+-----------------------+----------------------+------------------------------------+-------------------------------------------+-------------------------------------------------+--------------------------------------------------------+------------------------+---------------------+---

In [53]:
#Introduce PC2 table
sp_df_pc2 = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_item_status_last/*.parquet',
    header=True)
sp_df_pc2.show()

+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
|  item_skey|shop_skey|item_status_skey|item_status_weekly_skey|item_pc1_costs_before_cogs|item_pc2_costs_before_cogs|group_pc2_margin_simulated|group_pc2_margin_ratio_simulated|group_pc2_margin_simulated_pocket|group_pc2_margin_ratio_simulated_pocket|
+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
| 4184608796|        1|               4|                      4|                   674.169|                   800.340|                      null|                            null|                             null|                             

In [54]:
sp_df_item.createOrReplaceTempView('d_item_tmp')
sp_df_pc2.createOrReplaceTempView('d_item_pc2')

In [55]:
pc2_table = spark.sql(""" WITH pc2_avg as (
                            SELECT item_skey,
                            item_pc2_costs_before_cogs as pc2
                            from d_item_pc2
                            where shop_skey = 1)
                            SELECT pc.item_skey,di.item_code, pc.pc2
                            from d_item_tmp as di
                            INNER JOIN pc2_avg as pc on pc.item_skey = di.item_skey
                            where di.item_code like '0%'
                            """)
pc2_table.show()


+------------+------------------+-------+
|   item_skey|         item_code|    pc2|
+------------+------------------+-------+
|  4184608796|000000001000157424|800.340|
| 43499951512|000000001000114687|342.220|
| 63209271384|000000001000193143|113.050|
| 64339265064|000000001000038173|307.399|
| 67389826157|000000001000232523| 13.970|
|178212850851|000000001000220797|111.220|
|251843179141|000000001000125573|510.380|
|321153977917|000000001000151885|532.309|
|324513273733|000000001000176129| 29.980|
|360959333013|000000001000222621| 73.629|
|391563460770|000000001000061915|212.229|
|414751597565|000000001000024545| 65.040|
|451907443162|000000001000103142|285.139|
|453294840650|000000001000047001|146.060|
|462665822707|000000001000112387| 46.340|
|464745354617|000000001000020098|346.680|
|518044503882|000000001000016906|619.179|
|526193144479|000000001000010513| 33.609|
|551043742079|000000001000219102| 13.289|
|556925096750|000000001000187198| 15.800|
+------------+------------------+-

In [56]:
pc2_table.createOrReplaceTempView('pc2_table')

In [57]:
spark.sql("""SELECT COUNT(DISTINCT item_skey) FROM pc2_table""").show()
spark.sql("""SELECT COUNT(item_skey) FROM pc2_table""").show()

+-------------------------+
|count(DISTINCT item_skey)|
+-------------------------+
|                   229627|
+-------------------------+

+----------------+
|count(item_skey)|
+----------------+
|          229627|
+----------------+



In [58]:
spark_pc2_final = spark.sql(""" SELECT sp.*, pc.pc2, pc.item_skey
                                from spark_final_tmp as sp
                                 INNER JOIN pc2_table as pc on sp.item_code = pc.item_code
                                 """)

spark_pc2_final.show()

+------------------+--------------------+----------+----------+------------------+-------+------+---------------+-------+---------------+
|         item_code|       avg_conv_rate|last_price| last_date|  avg_unique_views|     PE|PE_err|has_enough_stat|    pc2|      item_skey|
+------------------+--------------------+----------+----------+------------------+-------+------+---------------+-------+---------------+
|000000001000012530|0.016219531350040728|     99.99|2021-02-13| 66.08138736263736|-0.0187|0.0059|           true| 44.680|247523697279378|
|000000001000012794|0.014916173111257461|    259.99|2021-02-25| 41.52597402597403| -0.012|0.0052|          false|141.050|151936474932722|
|000000001000021922|0.023490245459702064|    114.99|2021-02-16| 90.11309523809524|-0.0194|0.0025|           true|   null|156995084198890|
|000000001000022179|0.003121343752816...|    479.99|2021-02-06| 93.25873910040576|-0.0136|0.0033|           true|276.870| 11954970507506|
|000000001000024320|0.024532024531

In [59]:
df_final_spark = spark_pc2_final.toPandas()

In [60]:
df_final_spark.to_csv('./pickle_files/df_final_spark_top5k_080421.csv')

In [133]:
import math
import numpy as np


In [134]:
df_final_spark=df_final_spark.astype({'pc2': 'float64'})

print(df_final_spark.dtypes)

df_final_spark.dropna(inplace=True)
df_final_spark = df_final_spark[df_final_spark['PE']< 1.0]
df_final_spark = df_final_spark[df_final_spark['PE_err']<1.0]

item_code            object
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object


In [135]:
VAT = 1.19



In [136]:
def create_auxiliary_var(dataf=df_final_spark):
    dataf['Delta_price'] = (dataf['last_price']/VAT - dataf['pc2'])
    dataf['Margin_base'] = dataf['avg_unique_views']*dataf['avg_conv_rate']*(dataf['Delta_price'])

def classify(dataf= df_final_spark):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up',
                                        np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral'))



In [137]:
def create_Margin( increment = 0.01, dataf = df_final_spark, flag = 'up'):
    if flag == 'down':
        increment = -increment
        
    price          = 'price_'+flag
    Delta_price    = 'Delta_price_'+flag
    conv_rate      = 'conv_rate_'+flag
    conv_rate_err  = 'conv_rate_err_'+flag
    Margin         = 'Margin_'+flag
    Margin_err     = 'Margin_err_'+flag
    
    dataf[price]              =  dataf['last_price']*math.exp(increment)
    dataf[Delta_price]        =  (dataf[price]/VAT - dataf['pc2']) #MEVI I am overwriting this variable
    dataf[conv_rate]          =  dataf['avg_conv_rate']*np.exp(dataf['PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[conv_rate_err]      =  np.fabs(dataf[conv_rate])*np.fabs(dataf['PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[Margin]             =  dataf['avg_unique_views']*dataf[conv_rate]*(dataf[Delta_price])
    #Propagate Margin error
    dataf[Margin_err]          =  dataf['avg_unique_views']*dataf[conv_rate_err]*(dataf[Delta_price])



In [138]:
print(df_final_spark[['item_code','avg_conv_rate','last_price']].head())
#create_Margin(0.01)

            item_code  avg_conv_rate  last_price
0  000000001000012530       0.016220       99.99
1  000000001000012794       0.014916      259.99
3  000000001000022179       0.003121      479.99
4  000000001000024320       0.024532      119.99
5  000000001000034743       0.005481      279.99


In [139]:
original_df= df_final_spark
print(len(original_df.index))
tmp_df = pd.DataFrame()

original_df_up   = pd.DataFrame()
original_df_down = pd.DataFrame()

4155


In [140]:
create_auxiliary_var(original_df)
for inc in [0.01,0.02]: #np.linspace(0.01,0.1,10):
    print('Increment = ' + str(inc))
    if inc == 0.01:
        create_Margin(inc,original_df,'up')
        create_Margin(inc,original_df,'down')
        classify(original_df)
    #print(original_df)
        tmp_df           = tmp_df.append(original_df[original_df['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up.append(original_df[original_df['Group_flag'] == 'up'])
        original_df_down = original_df_down.append(original_df[original_df['Group_flag'] == 'down'])
        #print('tmp_DF')
        #print(tmp_df[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag', 'Delta_price']].head(20))
        #print('original_DF_up')
        #print(original_df_up[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag','Delta_price']].head(20))
        #print('original_DF_down')
        #print(original_df_down[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag','Delta_price']].head(20))
        #print('original_DF')
        #print(original_df[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_up_err','Margin_down','Margin_down_err','Group_flag','Delta_price']].head(20))
    if inc > 0.01:
        create_Margin(inc,original_df_up,'up')
        create_Margin(inc,original_df_down,'down')
        classify(original_df_up)
        classify(original_df_down)
        #print('tmp_DF')
        tmp_df           = tmp_df.append(original_df_up[original_df_up['Group_flag'] == 'neutral'])
        tmp_df           = tmp_df.append(original_df_down[original_df_down['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up[original_df_up['Group_flag'] == 'up']
        original_df_down = original_df_down[original_df_down['Group_flag'] == 'down']
        #print(tmp_df[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag', 'Delta_price']].head(20))
        #print('original_DF_up')
        #print(original_df_up[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag','Delta_price']].head(20))
        #print('original_DF_down')
        ##print(original_df_down[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_down','Group_flag','Delta_price']].head(20))
        #print('original_DF')
        #print(original_df[['item_code','last_price','PE', 'has_enough_stat','price_up','price_down', 'Margin_up','Margin_up_err','Margin_down','Margin_down_err','Group_flag','Delta_price']].head(20))
    #original_df['Margin_base'] = np.where(original_df['Group_flag'] == 'up', original_df['Margin_up'],  original_df['Margin_down'])
    #print(original_df[['item_code','avg_conv_rate','conv_rate_down','last_price','price_down','Margin_base','Margin_down']].head())
    original_df_down['Margin_base']   = original_df_down['Margin_down']
    original_df_up['Margin_base']   = original_df_up['Margin_up']
    #print(len(original_df.index))
    print(len(tmp_df.index))
    print(len(original_df_up.index))
    print(len(original_df_down.index))

Increment = 0.01
0
4155
0
Increment = 0.02
1
4154
0


In [141]:
df_final_spark.head(400)


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,conv_rate_err_up,Margin_up,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,Margin_down,Margin_err_down,Group_flag
0,000000001000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378,...,9.567734e-07,43.067609,2.540989e-03,98.995083,38.509145,0.016223,9.571313e-07,41.282173,2.435648e-03,up
1,000000001000012794,0.014916,259.99,2021-02-25,41.525974,-0.0120,0.0052,False,141.050,151936474932722,...,7.755479e-07,49.314333,2.564345e-03,257.403056,75.255089,0.014918,7.757341e-07,46.619245,2.424201e-03,up
3,000000001000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506,...,1.029903e-07,37.993099,1.253772e-03,475.214020,122.469512,0.003122,1.030184e-07,35.654815,1.176609e-03,up
4,000000001000024320,0.024532,119.99,2020-12-01,22.354839,-0.0220,0.0097,False,65.109,31827848594342,...,2.379083e-06,20.142108,1.953784e-03,118.796080,34.719638,0.024537,2.380130e-06,19.044767,1.847342e-03,up
5,000000001000034743,0.005481,279.99,2021-02-04,110.202494,-0.0186,0.0048,True,171.030,206871238538442,...,2.630578e-07,40.235416,1.931300e-03,277.204053,61.914582,0.005482,2.631557e-07,37.407263,1.795549e-03,up
6,000000001000034762,0.001473,599.99,2021-02-04,88.047619,-0.0030,0.0023,False,331.000,192085517288465,...,3.387139e-08,23.114149,5.316254e-04,594.020000,168.176470,0.001473,3.387342e-08,21.807911,5.015820e-04,up
7,000000001000119742,0.002190,349.99,2021-02-05,28.210938,-0.0247,0.0100,False,179.860,118864457774015,...,2.189260e-07,7.238711,7.238711e-04,346.507541,111.322808,0.002190,2.190341e-07,6.878812,6.878812e-04,up
8,000000001000125896,0.007258,829.99,2021-01-30,71.685847,-0.0055,0.0038,False,328.880,168012620788218,...,2.757723e-07,195.400790,7.425230e-03,821.731462,361.650640,0.007258,2.758026e-07,188.164379,7.150246e-03,up
9,000000001000127678,0.000122,1449.99,2021-02-25,58.300866,-0.0031,0.0024,False,987.679,10396424081065,...,2.930886e-09,1.730418,4.153003e-05,1435.562358,218.675923,0.000122,2.931068e-09,1.557007,3.736817e-05,up
10,000000001000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,280637889289282,...,1.218257e-07,52.564994,1.314125e-03,287.104551,52.445329,0.004874,1.218467e-07,48.103655,1.202591e-03,up


In [131]:
tmp_df[['item_code','avg_conv_rate','last_price','PE','PE_err','has_enough_stat','pc2','price_up','price_down','Margin_base','Margin_up','Margin_err_up','Margin_down','Margin_err_down','Group_flag']].head(400)


,item_code,avg_conv_rate,last_price,PE,PE_err,has_enough_stat,pc2,price_up,price_down,Margin_base,Margin_up,Margin_err_up,Margin_down,Margin_err_down,Group_flag
2602,000000001000227560,0.001835,779.99,-0.5212,0.3953,False,404.139,1.285986e+03,772.228970,2.068035e+01,4.202143e+01,8.305535e+00,19.834838,7.840711e-02,neutral
34,000000001000168702,0.052631,51.99,0.0846,0.2011,False,37.100,1.413235e+02,51.472691,6.424681e+01,1.566806e+02,3.150846e+01,10.841371,2.180200e-02,neutral
2677,000000001000188224,0.020552,519.99,-0.4047,0.2807,False,338.740,1.413479e+03,514.816013,1.597338e+02,2.902266e+02,8.146661e+01,48.292832,1.355580e-01,neutral
21,000000001000022565,0.023989,149.99,-0.1456,0.1268,False,111.680,1.108285e+03,148.497575,1.234077e+02,3.786547e+02,9.602682e+01,8.114193,1.028880e-02,neutral
2193,000000001000114706,0.143048,32.99,-0.2224,0.1132,False,17.530,2.437650e+02,32.661744,2.219651e+02,5.756125e+02,1.303187e+02,47.651036,5.394097e-02,neutral
2404,000000001000080476,0.092382,64.99,0.3766,0.1807,False,37.649,4.802148e+02,64.343339,6.884509e+02,3.312988e+03,1.197314e+03,69.746163,1.260313e-01,neutral
2598,000000001000173897,0.322786,19.99,0.0285,0.1260,False,12.180,1.477072e+02,19.791096,2.509976e+02,8.634318e+02,2.175848e+02,32.420836,4.085025e-02,neutral
3519,000000001000215686,0.069444,39.99,-0.1611,0.1465,False,26.870,2.954884e+02,39.592093,3.729968e+02,1.090395e+03,3.194859e+02,43.569581,6.382944e-02,neutral
40,000000001000185455,0.023306,129.99,0.0175,0.0981,False,39.430,2.863220e+06,128.696578,6.727732e+02,2.425326e+06,2.379245e+06,58.138645,5.703401e-02,neutral
88,000000001000215684,0.030163,63.99,-0.0397,0.0399,False,38.780,1.409474e+06,63.353289,1.174238e+03,2.823366e+06,1.126523e+06,51.282442,2.046169e-02,neutral


In [ ]:
tmp_df = tmp_df.sort_index()

In [ ]:
tmp_df['Margin_last'] = tmp_df['Margin_base']
tmp_df.head()

In [ ]:
create_Margin(0.01,df_final_spark)
df_final_spark.head()

In [ ]:
tmp_df['Margin_base']=df_final_spark['Margin_base']

tmp_df['Margin_rate_to_base'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_base'])
#tmp_df['Margin_rate_to_last'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_last'])

In [ ]:
tmp_df[['item_code','avg_conv_rate'	,'last_price','avg_unique_views', 'price_down', 'Delta_price', 'Margin_down','Margin_base','Margin_down_err','Margin_last','Group_flag','Margin_rate_to_base']]

In [ ]:
print(tmp_df['Margin_rate_to_base'].mean()*100)

In [ ]:
tmp_df.to_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2_no_stock_no_stat_filter_20210226.csv')